In [1]:
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-h0dx0051
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-h0dx0051
  Resolved https://github.com/huggingface/transformers.git to commit c817bc44e2915593a292df9e7d4e2c0dfefb6620
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7262017 sha256=9b931b8e538d7470013ac2bbe64007dcf896d81c966f5e7eb0e7e5241f53db1a
  Stored in directory: /tmp/pip-ephem-wheel-cache-e2lgbyd3/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.1
    Uninstalling transformers-4.30.1:
      Successfully uninstalled transformers-4.3

In [2]:
import pandas as pd
import numpy as np
import json
    
# with open('/kaggle/input/e2eqa-wiki-zalo-ai/processed/zac2022_train_merged_final.json', 'r') as f:
#     data = json.load(f)

# df = pd.DataFrame(data['data'])
# df.drop(columns='dirty_text', inplace=True)
# df.fillna('', inplace=True)
# df.head()

df = pd.DataFrame()

all_text = []
all_id = []
all_question = []
all_title = []
all_category = []
all_is_long = []

all_dirty_text = []


short_candidate_start = []
short_candidate = []
answer = []


with open('/kaggle/input/e2eqa-wiki-zalo-ai/processed/zac2022_train_merged_final.json', 'r') as f:
    data = json.load(f)


for item in data['data']:
    all_id.append(item['id'])
    all_question.append(item['question'])
    
    all_text.append(item['text'])
    all_title.append(item['title'])
    all_category.append(item['category'])
    all_is_long.append(item['is_long_answer'])
    
    if 'short_candidate_start' in item.keys():
        
        short_candidate_start.append(item['short_candidate_start'])
        
    else:
        short_candidate_start.append('')
    
    if 'short_candidate' in item.keys():
        
        short_candidate.append(item['short_candidate'])
    else:
         short_candidate.append('')
            
    if 'answer' in item.keys():
        
        answer.append(item['answer'])
    else:
        answer.append('')


    
df['id'] = all_id
df['text'] = all_text
df['title'] = all_title
df['question'] = all_question
df['category'] = all_category
df['is_long_answer'] = all_is_long
df['short_candidate_start'] = short_candidate_start
df['answer'] = answer
df['short_candidate'] = short_candidate


df.head()

,id,text,title,question,category,is_long_answer,short_candidate_start,answer,short_candidate
0,718d41cd997b2b44b0685ac54aa55bd8,Thủ tướng Trung Quốc là nhân vật lãnh đạo chín...,Trung Quốc,Thủ tướng Trung Quốc là gì,PARTIAL_ANNOTATION,True,,,
1,c926e7b0717202618a10dd907d4b4c39,"có 23 quốc gia không có lực lượng quân đội, ba...",,Đất nước nào không có quân đội,FULL_ANNOTATION,True,53,wiki/Danh_sách_quốc_gia_không_có_lực_lượng_vũ_...,"Costa Rica, Iceland, Panama, Micronesia, Quần ..."
2,d38ef5bf1fb82b410026ed82c8a44cae,Raymondienne (hay Raymonde Dien) sinh ngày 13 ...,Raymondienne,Pháp tấn công xâm lược Việt Nam vào ngày tháng...,FALSE_LONG_ANSWER,False,,,
3,b6b5589a98fdccd208dc752bac853993,Cúp cờ vua thế giới là tên gọi một số giải đấu...,Cúp cờ vua thế giới,Cờ vua còn có tên gọi nào khác,FALSE_LONG_ANSWER,False,,,
4,82396a18fa9812bfec4d3ecb7ae60905,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,Shkhara,Núi nào cao nhất châu âu,FULL_ANNOTATION,True,73,wiki/Elbrus,núi Elbrus


In [3]:
df.shape

(20857, 9)

In [4]:
with open('/kaggle/input/e2eqa-wiki-zalo-ai/processed/zac2022_train_merged_final.json', 'r') as f:
    data = json.load(f)

In [5]:
data['data'][4]['text'][73:]

'núi Elbrus , ngọn núi cao nhất của châu Âu .'

In [6]:
df_full_anno = df[df['category'] == 'FULL_ANNOTATION']

df_full_anno

,id,text,title,question,category,is_long_answer,short_candidate_start,answer,short_candidate
1,c926e7b0717202618a10dd907d4b4c39,"có 23 quốc gia không có lực lượng quân đội, ba...",,Đất nước nào không có quân đội,FULL_ANNOTATION,True,53,wiki/Danh_sách_quốc_gia_không_có_lực_lượng_vũ_...,"Costa Rica, Iceland, Panama, Micronesia, Quần ..."
4,82396a18fa9812bfec4d3ecb7ae60905,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,Shkhara,Núi nào cao nhất châu âu,FULL_ANNOTATION,True,73,wiki/Elbrus,núi Elbrus
9,ee8a272bcefac7b96037a1571984b1f8,"Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , v...",Alexandria,Thành phố nào là thủ phủ của Ai Cập trong đế q...,FULL_ANNOTATION,True,48,wiki/Alexandria,Alexandria
16,c51604b25a641b2665c4ce1784b8cc6a,Các nghiên cứu lịch sử cho thấy Cổ Am rất có t...,Cổ Am,Ai là người đứng đầu trong cuộc chống lại chín...,FULL_ANNOTATION,True,65,wiki/Lê_Chân,Lê Chân
23,04bf774a424133d27c34895960937b87,"Phục Hưng (tiếng Pháp: ""Renaissance"", , , từ ""...",Phục Hưng,Nền văn hoá Phục Hưng bắt nguồn từ nước nào,FULL_ANNOTATION,True,205,wiki/Ý,Ý
...,...,...,...,...,...,...,...,...,...
20826,be7f71e3927632f3f3f2afa68ff1b91b,Tiền giấy xuất hiện . Năm 1396 thời Trần Thuận...,Tiền tệ Đại Việt thời Trần,vua nào cho phát hành tiền giấy đầu tiên ở việ...,FULL_ANNOTATION,True,54,wiki/Hồ_Quý_Ly,Hồ Quý Ly
20836,5769cee0487a9674306d271854244b39,Máy quay phim: Phát minh kỳ diệu của anh em Lu...,,Ai là người phát minh ra máy quay phim,FULL_ANNOTATION,True,37,wiki/Auguste_và_Louis_Lumière,anh em Lumière Phương Hiền
20840,e47529f7ade3caaaa60096137798c6ff,Ngũ Hành Sơn hay núi Non Nước là một danh thắn...,Ngũ Hành Sơn,tên quận nào ở đà nẵng cũng là tên một danh th...,FULL_ANNOTATION,True,285,wiki/Ngũ_Hành_Sơn_(quận),quận Ngũ Hành Sơn
20842,935572f75515f8d8ab831648589899c6,Trận Như Nguyệt là một trận đánh lớn diễn ra ở...,Trận Như Nguyệt,trận đánh nào có ý nghĩa quyết định trong khán...,FULL_ANNOTATION,True,0,wiki/Trận_Như_Nguyệt,Trận Như Nguyệt


In [7]:
import json
from glob import glob
from tqdm import tqdm
import re
from nltk import word_tokenize as lib_tokenizer

dict_map = dict({})

def word_tokenize(text):
    global dict_map
    words = text.split()
    words_norm = []
    for w in words:
        if dict_map.get(w, None) is None:
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"')
        words_norm.append(dict_map[w])
    return words_norm

def strip_answer_string(text):
    text = text.strip()
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`':
        if text[0] != '(' and text[-1] == ')' and '(' in text:
            break
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1:
            break
        text = text[:-1].strip()
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`':
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1:
            break
        text = text[1:].strip()
    text = text.strip()
    return text


def strip_context(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [8]:
import string
df_partial_pseudo = pd.read_csv('/kaggle/input/e2eqa-wiki-zalo-ai/pseudo-data/df_za2022_partial_anno_pseudo_label.csv')

def post_process(x):
    return x.replace(",","").replace(".","").replace("?","").replace("!","").replace(")","").replace("(","")

df_partial_pseudo["pred_answer"] = df_partial_pseudo["pred_answer"].apply(post_process)

df_partial_pseudo = df_partial_pseudo[df_partial_pseudo['pred_answer_score'] > 0.6] 

df_partial_pseudo

,id,question,title,text,pred_answer,pred_answer_start,pred_answer_end,pred_answer_score
0,718d41cd997b2b44b0685ac54aa55bd8,Thủ tướng Trung Quốc là gì,Trung Quốc,Thủ tướng Trung Quốc là nhân vật lãnh đạo chín...,nhân vật lãnh đạo chính phủ,24,51,0.991081
1,e27bcf72913542ee9de69fac59ad77a9,Tỉnh Đắk Nông được thành lập vào ngày tháng nă...,Đắk Nông,Đắk Nông hay Đắc Nông là một tỉnh ở Tây Nguyên...,ngày 1 tháng 1 năm 2004,91,114,0.999969
2,b8267523a48de0d700116acdbce2ee95,Tỉnh nào của nước ta vừa giáp Lào vừa giáp Cam...,Tây Nguyên,"Tây Nguyên là vùng cao nguyên, phía bắc giáp t...",Kon Tum,289,296,0.999406
4,253090162cf3fc0dd5e8537b53bcba99,hải dương có bao nhiêu huyện,Lịch sử hành chính Hải Dương,"Cùng năm , thành lập một số thị trấn thuộc các...",8,209,210,0.872943
6,155b65ee96bab3f74cafec51616e330f,Hari Won sinh năm bao nhiêu,Hari Won,"Hari Won ( 하리원 , "" Hari Nguyễn "" ) tên thật là...",1985,110,114,0.971467
...,...,...,...,...,...,...,...,...
3491,d701dfcba59420604c66e803c9556754,vì sao sao Hỏa có màu đỏ,Sao Hoả,"Sao Hoả còn gọi là : Hoả Tinh , ( Tiếng Anh : ...",sắt ôxít có mặt rất nhiều trên bề mặt hành tinh,222,269,0.679995
3494,1514053e80d5aadd764f4f5f916df004,Thời đại Hồng Bàng còn có tên gọi khác là gì,Hồng Bàng,Hồng Bàng Thị hay Thời đại Hồng Bàng là một gi...,Hồng Bàng Thị,0,13,0.999944
3495,93c746695c50932ac45ac498a192a3e5,Hà Nội có bao nhiêu đơn vị hành chính,Lịch sử hành chính Hà Nội,"Vào thời điểm hiện tại ( 2017 ) , về mặt hành ...",30,75,77,0.999760
3496,278ad127825c085a54fa22116c281f92,Ngày thành lập Google,Google,Tên miền www.google.com được đăng ký ngày 15 t...,ngày 7 tháng 9 năm 1998,114,137,0.999846


In [9]:
df_za2019_pseudo = pd.read_csv('/kaggle/input/e2eqa-wiki-zalo-ai/pseudo-data/df_za_2019_pseudo_label.csv')


df_za2019_pseudo["pred_answer"] = df_za2019_pseudo["pred_answer"].apply(post_process)

df_za2019_pseudo = df_za2019_pseudo[df_za2019_pseudo['pred_answer_score'] > 0.6] 


df_za2019_pseudo

,id,question,title,text,pred_answer,pred_answer_start,pred_answer_end,pred_answer_score
0,1558001276_0,Nơi Trần Quốc Toàn đại phá quân Mông -Nguyên t...,NaN,"Hàm Tử là một xã thuộc huyện Khoái Châu, tỉnh ...",Hưng Yên,46,55,0.999635
1,1554518024_0,Ban chỉ đạo trung ương về phòng chống tham nhũ...,"Ban Chỉ đạo Trung ương về phòng, chống tham nh...","Ngày 1 tháng 2 năm 2013, Bộ Chính trị ban hành...",Ngày 1 tháng 2 năm 2013,0,24,0.999978
2,1557393113_0,Quê mẹ của Bác Hồ là làng gì,NaN,"Làng Hoàng Trù cách làng Kim Liên khoảng 2 km,...",Làng Hoàng Trù,0,14,0.917361
4,1554702900_0,Việt Nam đã đặt quan hệ ngoại giao với tất cả ...,Quan hệ ngoại giao của Việt Nam,"Việt Nam hiện nay, dưới chính thể Cộng hòa xã ...",188 nước,98,106,0.999102
5,1554823810_2,Tỉnh nào ở điểm cực nam của Việt Nam,Đất Mũi,"Đất Mũi là một xã thuộc huyện Ngọc Hiển, tỉnh ...",Cà Mau,46,53,0.999941
...,...,...,...,...,...,...,...,...
381,1554453817_1,Bác Hồ ra đi cứu nước vào ngày tháng năm nào,Ngày Bác Hồ ra đi tìm đường cứu nước,Ngày Bác Hồ ra đi tìm đường cứu nước (ngày 5 t...,ngày 5 tháng 6 năm 1911,37,62,0.999999
387,1566783571_7,chủ tịch ủy ban nhân dân của Hải Phòng là ngườ...,Lê Đức Thịnh,Lê Đức Thịnh ( 1924 – 2001 ) là nhà cách mạng ...,Lê Đức Thịnh,0,12,0.999970
389,1553071483_2,Vị tướng đầu tiên được Bác Hồ phong tên là gì,Lê Thiết Hùng,Thiếu tướng Lê Thiết Hùng (1908 – 1986) nhà ho...,Lê Thiết Hùng,12,25,0.999836
392,1554448829_0,Ngoài trái đất hành tinh nào có khả năng xuất ...,Sự sống ngoài Trái Đất,"Có ý kiến cho rằng Alpha Centauri, chòm sao gầ...",Alpha Centauri,19,34,0.999897


In [10]:
with open('/kaggle/input/e2eqa-wiki-zalo-ai/zac2022_val.json/zac2022_val.json', 'r') as f:
    val = json.load(f)

all_valid_id = []

for item in val['data']:
    all_valid_id.append(item['id'])
    
    
len(val['data']), len(all_valid_id)

(749, 749)

In [11]:
all_train = []

for item in data['data']:
    if item['id'] not in all_valid_id:
        all_train.append(item)

train_data = {'data': all_train}

len(train_data['data']), len(val['data'])

(20107, 749)

In [12]:
def handle_file(data, train = True):
    
    qa_data = data['data']
    
    norm_samples = []
    
    ## za 2019 data set pseudo label
    
    if train ==True:
        for i, row in df_za2019_pseudo.iterrows():

            print('zalo2019 pseudo dataset!')

            context_raw = row['text']
            question = row['question']

            answer_raw = row['pred_answer']
            answer_index_raw = row['pred_answer_start']


            if context_raw[answer_index_raw: answer_index_raw + len(answer_raw)] == answer_raw:

        #                 print('True!!!!!!!!!')

                context_prev = strip_context(context_raw[:answer_index_raw])
                answer = strip_answer_string(answer_raw)
                context_next = strip_context(context_raw[answer_index_raw + len(answer):])

                context_prev = ' '.join(word_tokenize(context_prev))
                context_next = ' '.join(word_tokenize(context_next))
                answer = ' '.join(word_tokenize(answer))
                question = ' '.join(word_tokenize(question))

                context = "{} {} {}".format(context_prev, answer, context_next).strip()

                norm_samples.append({
                    "context": context,
                    "question": question,
                    "answer_text": answer,
                    "answer_start_idx": len("{} {}".format(context_prev, answer).strip()) - len(answer)
                })

    #za2022 dataset

    for item in tqdm(qa_data, total=len(qa_data)):
        
        #preprocess
        
        if 'dirty_text' in item.keys():
            if item['dirty_text'] != None:
                
                context_raw = item['dirty_text']
            else:
                
                context_raw = item['text']
            
#             print(context_raw)
        else:
            context_raw = item['text']
            
#         print(context_raw)
            
        id_ = item['id']
        category = item['category']
        is_long_answer = item['is_long_answer']
        question = item['question']
        
        ## train / valid
        
        
        
        if category == 'FULL_ANNOTATION':
            
            answer_raw = item['short_candidate']
            
            if 'dirty_text' in item.keys():
                print('use dirty text !!!')
                if item['dirty_text'] != None:
                    answer_index_raw = context_raw.find(answer_raw)
                else:
                    answer_index_raw = item['short_candidate_start']
                    
            else:
                answer_index_raw = item['short_candidate_start']
            
            if context_raw[answer_index_raw: answer_index_raw + len(answer_raw)] == answer_raw:
                
                print('True!!!!!!!!!')
                
                context_prev = strip_context(context_raw[:answer_index_raw])
                answer = strip_answer_string(answer_raw)
                context_next = strip_context(context_raw[answer_index_raw + len(answer):])

                context_prev = ' '.join(word_tokenize(context_prev))
                context_next = ' '.join(word_tokenize(context_next))
                answer = ' '.join(word_tokenize(answer))
                question = ' '.join(word_tokenize(question))

                context = "{} {} {}".format(context_prev, answer, context_next).strip()

                norm_samples.append({
                    "context": context,
                    "question": question,
                    "answer_text": answer,
                    "answer_start_idx": len("{} {}".format(context_prev, answer).strip()) - len(answer)
                })
                
        elif category == 'PARTIAL_ANNOTATION':
            
            if id_ in df_partial_pseudo['id'].unique():
                
                print('pseudo data!!')
                
                answer_raw = df_partial_pseudo[df_partial_pseudo['id'] == id_]['pred_answer'].values[0]
                answer_index_raw = df_partial_pseudo[df_partial_pseudo['id'] == id_]['pred_answer_start'].values[0] # item['short_candidate_start']

                if context_raw[answer_index_raw: answer_index_raw + len(answer_raw)] == answer_raw:

    #                 print('True!!!!!!!!!')

                    context_prev = strip_context(context_raw[:answer_index_raw])
                    answer = strip_answer_string(answer_raw)
                    context_next = strip_context(context_raw[answer_index_raw + len(answer):])

                    context_prev = ' '.join(word_tokenize(context_prev))
                    context_next = ' '.join(word_tokenize(context_next))
                    answer = ' '.join(word_tokenize(answer))
                    question = ' '.join(word_tokenize(question))

                    context = "{} {} {}".format(context_prev, answer, context_next).strip()

                    norm_samples.append({
                        "context": context,
                        "question": question,
                        "answer_text": answer,
                        "answer_start_idx": len("{} {}".format(context_prev, answer).strip()) - len(answer)
                    })
                    
            else:
                context_raw = ' '.join(word_tokenize(context_raw))
                question = ' '.join(word_tokenize(question))
                norm_samples.append({
                    "context": context_raw,
                    "question": question,
                    "answer_text": '',
                    "answer_start_idx": 0
                })
                    
                
        else:
            context_raw = ' '.join(word_tokenize(context_raw))
            question = ' '.join(word_tokenize(question))
            norm_samples.append({
                "context": context_raw,
                "question": question,
                "answer_text": '',
                "answer_start_idx": 0
            })
     
    
    print('Len training sample: ', len(norm_samples))
    return norm_samples

In [13]:
len(train_data['data'])

20107

In [14]:
dict_data_train = handle_file(train_data, train = True)

zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!
zalo2019 pseudo dataset!


  0%|          | 57/20107 [00:00<00:35, 567.36it/s]

pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!


  1%|          | 129/20107 [00:00<00:30, 653.71it/s]

pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!


...

pseudo data!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
pseudo data!!
pseudo data!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use

 99%|█████████▉| 19918/20107 [00:13<00:00, 1754.06it/s]

pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!


100%|██████████| 20107/20107 [00:13<00:00, 1488.64it/s]

pseudo data!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
pseudo data!!
use dirty text !!!
True!!!!!!!!!
use dir

In [15]:
with open('qa_full_anno_train.jsonl', 'w', encoding='utf-8') as file:
    for item in dict_data_train:
        file.write("{}\n".format(json.dumps(item, ensure_ascii=False)))

print("Total: {} samples".format(len(dict_data_train)))

Total: 19136 samples


In [16]:
dict_data_valid = handle_file(val, train = False)

 50%|█████     | 378/749 [00:00<00:00, 3760.47it/s]

use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!

100%|██████████| 749/749 [00:00<00:00, 3596.63it/s]

use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
use dirty text !!!
True!!!!!!!!!
Len training sample:  736


In [17]:
with open('qa_full_anno_valid.jsonl', 'w', encoding='utf-8') as file:
    for item in dict_data_valid:
        file.write("{}\n".format(json.dumps(item, ensure_ascii=False)))

print("Total: {} samples".format(len(dict_data_valid)))

Total: 736 samples


In [18]:
import datasets
from datasets import concatenate_datasets
import glob

# datasets.temp_seed(101)
datasets.disable_progress_bar()


def assert_sample(sample):
    assert sample['context'][sample['answer_start_idx']: sample['answer_start_idx'] + len(sample['answer_text'])] == \
           sample['answer_text'], sample
    assert len(sample['context']) > 0
    assert len(sample['question']) > 0
    return True


def format_sample(sample):
    context_prev = sample['context'][:sample['answer_start_idx']].split()
    sample['answer_word_start_idx'] = len(context_prev)
    sample['answer_word_end_idx'] = len(context_prev) + len(sample['answer_text'].split()) - 1
    return sample


if __name__ == "__main__":
    train_set = []
    valid_set = []
    for part in ['/kaggle/working/qa_full_anno_train.jsonl']:  #glob.glob('../data-bin/unify/*.jsonl'):
        dataset = datasets.load_dataset('json', data_files=[part])['train']
        dataset.filter(assert_sample)
        dataset = dataset.map(format_sample)

#         all_data = dataset.train_test_split(test_size=0.1)
#         train = all_data['train']
#         valid = all_data['test']
#         train_set.append(train)
#         valid_set.append(valid)
        train_set.append(dataset)
        
    for part in ['/kaggle/working/qa_full_anno_valid.jsonl']:  #glob.glob('../data-bin/unify/*.jsonl'):
        dataset = datasets.load_dataset('json', data_files=[part])['train']
        dataset.filter(assert_sample)
        dataset = dataset.map(format_sample)

#         all_data = dataset.train_test_split(test_size=0.1)
#         train = all_data['train']
#         valid = all_data['test']
#         train_set.append(train)
#         valid_set.append(valid)
        valid_set.append(dataset)

    train_dataset = concatenate_datasets(train_set)
    valid_dataset = concatenate_datasets(valid_set)

    train_dataset.save_to_disk('/kaggle/working/data-bin/processed/train.dataset')
    valid_dataset.save_to_disk('/kaggle/working/data-bin/processed/valid.dataset')

    print("Train: {} samples".format(len(train_dataset)))
    print("Valid: {} samples".format(len(valid_dataset)))


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-1d983b87dcd55a7f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-cd981311ec3bbf65/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.
Train: 19136 samples
Valid: 736 samples


In [19]:
%%writefile train.py 

import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import TrainingArguments
# from model.mrc_model import MRCQuestionAnswering
from transformers import Trainer
# from utils import data_loader
import numpy as np
from datasets import load_metric
import os

import datasets
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import torch
import numpy as np
from nltk import word_tokenize

tokenizer = AutoTokenizer.from_pretrained("nguyenvulebinh/vi-mrc-large")

# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large",
# #                                           cache_dir='./model-bin/cache',
#                                           #local_files_only=True
#                                          )


def compute_metrics(eval_pred):
    metric = datasets.load_metric("squad", cache_dir='./log/metric')
    # print(eval_pred)
    logits, labels = eval_pred
    logits = list(zip(logits[0], logits[1]))
    labels, span_ids, samples_input_ids, word_lengths = list(zip(labels[0], labels[1])), labels[2], labels[3], labels[4]
    predictions = []
    references = []
    for idx, (predict, span_truth, input_ids, sample_words_length) in enumerate(
            list(zip(logits, span_ids, samples_input_ids, word_lengths))):
        span_truth = np.delete(span_truth, np.where(span_truth == -100))
        input_ids = np.delete(input_ids, np.where(input_ids == -100))

        # Get the most likely beginning of answer with the argmax of the score
        answer_start = sum(sample_words_length[:np.argmax(predict[0])])
        # Get the most likely end of answer with the argmax of the score
        answer_end = sum(sample_words_length[:np.argmax(predict[1]) + 1])

        answer = tokenizer.convert_tokens_to_string(
            tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
        answer_truth = tokenizer.convert_tokens_to_string(
            tokenizer.convert_ids_to_tokens(span_truth))

        predictions.append({'prediction_text': answer, 'id': str(idx)})
        references.append({'answers': {'answer_start': [answer_start], 'text': [answer_truth]}, 'id': str(idx)})
    results = metric.compute(predictions=predictions, references=references)
    return results


def data_collator(samples):
    if len(samples) == 0:
        return {}

    # for sample in samples:
    #     start_idx = sum(sample['words_lengths'][:sample['start_idx']])
    #     end_idx = sum(sample['words_lengths'][:sample['end_idx'] + 1])
    #     if len(sample['answer_text']) > 0:
    #         answer_predict = set(
    #             list(' '.join(word_tokenize(tokenizer.decode(sample['input_ids'][start_idx:end_idx])))))
    #         answer_truth = set(list(' '.join(word_tokenize(sample['answer_text']))))
    #         if answer_predict != answer_truth:
    #             print("{}\n{}\n\n-------\n".format(
    #                 ' '.join(word_tokenize(tokenizer.decode(sample['input_ids'][start_idx:end_idx]))),
    #                 ' '.join(word_tokenize(sample['answer_text'])).replace('``', '"').replace("''", '"'),
    #                 sample))

    for sample in samples:
        start_idx = sum(sample['words_lengths'][:sample['start_idx']])
        end_idx = sum(sample['words_lengths'][:sample['end_idx'] + 1])
        sample['span_answer_ids'] = sample['input_ids'][start_idx:end_idx]

    def collate_tokens(values, pad_idx, eos_idx=None, left_pad=False, move_eos_to_beginning=False):
        """Convert a list of 1d tensors into a padded 2d tensor."""
        size = max(v.size(0) for v in values)
        res = values[0].new(len(values), size).fill_(pad_idx)

        def copy_tensor(src, dst):
            assert dst.numel() == src.numel()
            if move_eos_to_beginning:
                assert src[-1] == eos_idx
                dst[0] = eos_idx
                dst[1:] = src[:-1]
            else:
                dst.copy_(src)

        for i, v in enumerate(values):
            copy_tensor(v, res[i][size - len(v):] if left_pad else res[i][:len(v)])
        return res

    input_ids = collate_tokens([torch.tensor(item['input_ids']) for item in samples], pad_idx=tokenizer.pad_token_id)
    attention_mask = torch.zeros_like(input_ids)
    for i in range(len(samples)):
        attention_mask[i][:len(samples[i]['input_ids'])] = 1
    words_lengths = collate_tokens([torch.tensor(item['words_lengths']) for item in samples], pad_idx=0)
    answer_start = collate_tokens([torch.tensor([item['start_idx']]) for item in samples], pad_idx=0)
    answer_end = collate_tokens([torch.tensor([item['end_idx']]) for item in samples], pad_idx=0)
    span_answer_ids = collate_tokens([torch.tensor(item['span_answer_ids']) for item in samples],
                                     pad_idx=-100)

    batch_samples = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'words_lengths': words_lengths,
        'start_positions': answer_start,
        'end_positions': answer_end,
        'span_answer_ids': span_answer_ids
    }

    return batch_samples


def tokenize_function(example):
    example["question"] = example["question"].split()
    example["context"] = example["context"].split()
    # max_len_single_sentence = tokenizer.max_len_single_sentence
    max_len_single_sentence = 368

    question_sub_words_ids = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(w)) for w in example["question"]]
    context_sub_words_ids = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(w)) for w in example["context"]]
    valid = True
    if len([j for i in question_sub_words_ids + context_sub_words_ids for j in
            i]) > max_len_single_sentence - 1:
        question_ids = [j for i in question_sub_words_ids for j in i]
        context_ids = [j for i in context_sub_words_ids[:example['answer_word_end_idx'] + 1] for j in i]
        remain_tokens = max_len_single_sentence - 1 - len(question_ids)
        if len(question_ids + context_ids) < max_len_single_sentence - 1:
            context_sub_words_ids_revise = context_sub_words_ids[:example['answer_word_end_idx'] + 1]
            idx = example['answer_word_end_idx'] + 1
            while len([j for i in (context_sub_words_ids_revise + [context_sub_words_ids[idx]]) for j in
                       i]) < remain_tokens and idx < len(context_sub_words_ids):
                context_sub_words_ids_revise.append(context_sub_words_ids[idx])
                idx += 1
            context_sub_words_ids = context_sub_words_ids_revise
        else:
            valid = False

    question_sub_words_ids = [[tokenizer.bos_token_id]] + question_sub_words_ids + [[tokenizer.eos_token_id]]
    context_sub_words_ids = context_sub_words_ids + [[tokenizer.eos_token_id]]

    input_ids = [j for i in question_sub_words_ids + context_sub_words_ids for j in i]
    if len(input_ids) > max_len_single_sentence + 2:
        valid = False

    words_lengths = [len(item) for item in question_sub_words_ids + context_sub_words_ids]

    return {
        "input_ids": input_ids,
        "words_lengths": words_lengths,
        "start_idx": (example['answer_word_start_idx'] + len(question_sub_words_ids)) if len(
            example["answer_text"]) > 0 else 0,
        "end_idx": (example['answer_word_end_idx'] + len(question_sub_words_ids)) if len(
            example["answer_text"]) > 0 else 0,
        "valid": valid
    }


def get_dataloader(train_path, valid_path, batch_size=2, num_proc=10):
    train_set = datasets.load_from_disk(train_path)
    valid_set = datasets.load_from_disk(valid_path)
    print("Train set: ", len(train_set))
    print("Valid set: ", len(valid_set))
    # unique_tags = set(tag for doc in tags for tag in train_set)
    train_set = train_set.shuffle().map(tokenize_function, batched=False, num_proc=num_proc).filter(
        lambda example: example['valid'], num_proc=num_proc)
    valid_set = valid_set.map(tokenize_function, batched=False, num_proc=num_proc).filter(
        lambda example: example['valid'], num_proc=num_proc)
    # train_set = train_set.sort('src_ids_len')
    # valid_set = valid_set.sort('src_ids_len')

    print("Train set: ", len(train_set))
    print("Valid set: ", len(valid_set))
    return train_set, valid_set
    #return train_set, valid_set.shard(500, 0)
    #
    # train_dataloader = DataLoader(
    #     train_set, shuffle=True, batch_size=batch_size, collate_fn=data_collator
    # )
    # valid_dataloader = DataLoader(
    #     valid_set, batch_size=batch_size, collate_fn=data_collator
    # )
    #
    # return train_dataloader, valid_dataloader


def build_target_dictionary():
    data_set = datasets.load_from_disk('./data-bin/processed/train.dataset')
    labels = set([item['language'] for item in data_set])
    labels2id = {tag: idx for idx, tag in enumerate(labels)}
    # id2labels = {idx: tag for tag, idx in labels2id.items()}

    tags2id = {
        "same": 1,
        "change": 0
    }
    # id2tags = {idx: tag for tag, idx in tags2id.items()}

    return labels2id, tags2id


\
from transformers.models.roberta.modeling_roberta import *
_CHECKPOINT_FOR_DOC = "roberta-base"
_CONFIG_FOR_DOC = "RobertaConfig"
_TOKENIZER_FOR_DOC = "RobertaTokenizer"


class MRCQuestionAnswering(RobertaPreTrainedModel):
    config_class = RobertaConfig

#     def _reorder_cache(self, past, beam_idx):
#         pass

#     _keys_to_ignore_on_load_unexpected = [r"pooler"]
#     _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.qa_outputs = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
            self,
            input_ids=None,
            words_lengths=None,
            start_idx=None,
            end_idx=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            start_positions=None,
            end_positions=None,
            span_answer_ids=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        context_embedding = sequence_output

        # Compute align word sub_word matrix
        batch_size = input_ids.shape[0]
        max_sub_word = input_ids.shape[1]
        max_word = words_lengths.shape[1]
        align_matrix = torch.zeros((batch_size, max_word, max_sub_word))

        for i, sample_length in enumerate(words_lengths):
            for j in range(len(sample_length)):
                start_idx = torch.sum(sample_length[:j])
                align_matrix[i][j][start_idx: start_idx + sample_length[j]] = 1 if sample_length[j] > 0 else 0

        align_matrix = align_matrix.to(context_embedding.device)
        # Combine sub_word features to make word feature
        context_embedding_align = torch.bmm(align_matrix, context_embedding)

        logits = self.qa_outputs(context_embedding_align)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        total_loss = None
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions = start_positions.clamp(0, ignored_index)
            end_positions = end_positions.clamp(0, ignored_index)

            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2

        if not return_dict:
            output = (start_logits, end_logits) + outputs[2:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
if __name__ == "__main__":
    # tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
    model = MRCQuestionAnswering.from_pretrained("nguyenvulebinh/vi-mrc-large")
    
        #("xlm-roberta-large",
         #                                        cache_dir='./model-bin/cache',
                                                 #local_files_only=True
#                                                 )
    print(model)
    print(model.config)

    train_dataset, valid_dataset = get_dataloader(
        train_path='./data-bin/processed/train.dataset',
        valid_path='./data-bin/processed/valid.dataset'
    )

    training_args = TrainingArguments("model-bin/test",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs= 5, # 8, #10,
                                      learning_rate=1e-5, #1e-4
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      gradient_accumulation_steps=1,
                                      logging_dir='./log',
                                      logging_steps=5,
                                      label_names=['start_positions',
                                                   'end_positions',
                                                   'span_answer_ids',
                                                   'input_ids',
                                                   'words_lengths'],
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      metric_for_best_model='f1',
                                      load_best_model_at_end=True,
                                      save_total_limit=2,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      evaluation_strategy="epoch",
                                      )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()


Writing train.py


In [20]:
!python train.py

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso